In [3]:
import requests
import socket
import time

# IP addresses to try (in order of likelihood)
IP_ADDRESSES = [
    "10.255.255.254",  # Your current setting
    "host.docker.internal",  # Special DNS name that WSL2 can use to reach Windows
    "172.17.0.1",  # Another common WSL2->Windows IP
    "192.168.29.200",  # Common router IP (fallback if WSL is going through the router)
    "localhost"  # Try this as a sanity check
]

def test_connection(host):
    print(f"\n🔍 Testing connection to Ollama at {host}:11434")
    
    # Test socket connection first
    try:
        print(f"  Socket test: ", end="", flush=True)
        start = time.time()
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(3)
        result = sock.connect_ex((host, 11434))
        duration = time.time() - start
        
        if result == 0:
            print(f"✅ Success ({duration:.2f}s)")
            sock.close()
        else:
            print(f"❌ Failed - Port closed or unreachable ({duration:.2f}s)")
            sock.close()
            return False
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False
    
    # Test HTTP API
    try:
        print(f"  HTTP API test: ", end="", flush=True)
        start = time.time()
        response = requests.get(f"http://{host}:11434/api/tags", timeout=5)
        duration = time.time() - start
        
        if response.status_code == 200:
            print(f"✅ Success ({duration:.2f}s)")
            models = response.json().get("models", [])
            if models:
                print(f"  📋 Available models: {', '.join(m['name'] for m in models)}")
            else:
                print(f"  ⚠️ No models found")
            return True
        else:
            print(f"❌ Failed - Status code: {response.status_code} ({duration:.2f}s)")
            return False
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# Additional test: Try Ollama client directly
def test_ollama_client(host):
    print(f"\n🤖 Testing Ollama Python client with host {host}")
    try:
        import ollama
        print("  Client test: ", end="", flush=True)
        start = time.time()
        response = ollama.list(host=host)
        duration = time.time() - start
        print(f"✅ Success ({duration:.2f}s)")
        print(f"  📋 Models via client: {', '.join(m['name'] for m in response['models'])}")
        return True
    except ImportError:
        print("  ❌ ollama package not installed correctly")
        return False
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        return False

# Try all IP addresses
print("🔄 Testing Ollama connectivity from WSL to Windows")
print("🔄 Make sure Ollama is running on Windows!")

success = False
working_host = None

for host in IP_ADDRESSES:
    if test_connection(host):
        print(f"\n✅ SUCCESS: Connected to Ollama at {host}:11434")
        success = True
        working_host = host
        break
    else:
        print(f"\n❌ Failed to connect to {host}:11434")

if success:
    # Try the client too
    test_ollama_client(working_host)
    
    print(f"\n🎉 CONNECTION SUCCESSFUL!")
    print(f"Update your app.py with:")
    print(f"OLLAMA_HOST = \"{working_host}\"")
else:
    print("\n❌ All connection attempts failed!")
    print("\nTroubleshooting tips:")
    print("1. Verify Ollama is running on Windows (check Task Manager)")
    print("2. Check Windows Firewall settings - add exception for port 11434")
    print("3. Try manually accessing http://localhost:11434 in a Windows browser")
    print("4. Restart WSL with: 'wsl --shutdown' then reopen your terminal")

🔄 Testing Ollama connectivity from WSL to Windows
🔄 Make sure Ollama is running on Windows!

🔍 Testing connection to Ollama at 10.255.255.254:11434
  Socket test: ❌ Failed - Port closed or unreachable (0.00s)

❌ Failed to connect to 10.255.255.254:11434

🔍 Testing connection to Ollama at host.docker.internal:11434
  Socket test: ❌ Error: [Errno -2] Name or service not known

❌ Failed to connect to host.docker.internal:11434

🔍 Testing connection to Ollama at 172.17.0.1:11434
  Socket test: 

❌ Failed - Port closed or unreachable (3.01s)

❌ Failed to connect to 172.17.0.1:11434

🔍 Testing connection to Ollama at 192.168.29.200:11434
  Socket test: ✅ Success (0.00s)
  HTTP API test: ✅ Success (0.04s)
  📋 Available models: gemma3:12b, llama3.2:latest

✅ SUCCESS: Connected to Ollama at 192.168.29.200:11434

🤖 Testing Ollama Python client with host 192.168.29.200
  Client test:   ❌ Error: Client.list() got an unexpected keyword argument 'host'

🎉 CONNECTION SUCCESSFUL!
Update your app.py with:
OLLAMA_HOST = "192.168.29.200"


In [4]:
import requests
import socket
import time
import re

def get_windows_host_ip():
    """
    Retrieves the Windows host IP address from /etc/resolv.conf.
    """
    try:
        with open('/etc/resolv.conf', 'r') as file:
            content = file.read()
            # Regular expression to find the IP address after 'nameserver'
            match = re.search(r'nameserver\s+(\d+\.\d+\.\d+\.\d+)', content)
            if match:
                return match.group(1)
    except Exception as e:
        print(f"Error reading /etc/resolv.conf: {e}")
    return None

def test_connection(host):
    print(f"\n🔍 Testing connection to Ollama at {host}:11434")

    # Test socket connection first
    try:
        print(f"  Socket test: ", end="", flush=True)
        start = time.time()
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(3)
        result = sock.connect_ex((host, 11434))
        duration = time.time() - start

        if result == 0:
            print(f"✅ Success ({duration:.2f}s)")
            sock.close()
        else:
            print(f"❌ Failed - Port closed or unreachable ({duration:.2f}s)")
            sock.close()
            return False
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

    # Test HTTP API
    try:
        print(f"  HTTP API test: ", end="", flush=True)
        start = time.time()
        response = requests.get(f"http://{host}:11434/api/tags", timeout=5)
        duration = time.time() - start

        if response.status_code == 200:
            print(f"✅ Success ({duration:.2f}s)")
            models = response.json().get("models", [])
            if models:
                print(f"  📋 Available models: {', '.join(m['name'] for m in models)}")
            else:
                print(f"  ⚠️ No models found")
            return True
        else:
            print(f"❌ Failed - Status code: {response.status_code} ({duration:.2f}s)")
            return False
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# Additional test: Try Ollama client directly
def test_ollama_client(host):
    print(f"\n🤖 Testing Ollama Python client with host {host}")
    try:
        import ollama
        print("  Client test: ", end="", flush=True)
        start = time.time()
        response = ollama.list(host=host)
        duration = time.time() - start
        print(f"✅ Success ({duration:.2f}s)")
        print(f"  📋 Models via client: {', '.join(m['name'] for m in response['models'])}")
        return True
    except ImportError:
        print("  ❌ ollama package not installed correctly")
        return False
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        return False

# Main execution
print("🔄 Testing Ollama connectivity from WSL to Windows")
print("🔄 Make sure Ollama is running on Windows!")

# Retrieve the Windows host IP address
windows_host_ip = get_windows_host_ip()

if windows_host_ip:
    if test_connection(windows_host_ip):
        print(f"\n✅ SUCCESS: Connected to Ollama at {windows_host_ip}:11434")
        # Try the client too
        test_ollama_client(windows_host_ip)
        print(f"\n🎉 CONNECTION SUCCESSFUL!")
        print(f"Update your app.py with:")
        print(f"OLLAMA_HOST = \"{windows_host_ip}\"")
    else:
        print(f"\n❌ Failed to connect to {windows_host_ip}:11434")
        print("\nTroubleshooting tips:")
        print("1. Verify Ollama is running on Windows (check Task Manager)")
        print("2. Check Windows Firewall settings - add exception for port 11434")
        print("3. Try manually accessing http://localhost:11434 in a Windows browser")
        print("4. Restart WSL with: 'wsl --shutdown' then reopen your terminal")
else:
    print("\n❌ Unable to determine Windows host IP address from /etc/resolv.conf")
    print("\nTroubleshooting tips:")
    print("1. Ensure /etc/resolv.conf exists and contains the nameserver entry")
    print("2. Verify WSL2 is properly configured and running")
    print("3. Restart WSL with: 'wsl --shutdown' then reopen your terminal")


🔄 Testing Ollama connectivity from WSL to Windows
🔄 Make sure Ollama is running on Windows!

🔍 Testing connection to Ollama at 10.255.255.254:11434
  Socket test: ❌ Failed - Port closed or unreachable (0.00s)

❌ Failed to connect to 10.255.255.254:11434

Troubleshooting tips:
1. Verify Ollama is running on Windows (check Task Manager)
2. Check Windows Firewall settings - add exception for port 11434
3. Try manually accessing http://localhost:11434 in a Windows browser
4. Restart WSL with: 'wsl --shutdown' then reopen your terminal
